# Olympics Data (Python + SQL)

Let's analzye some olympics data. I'll do some cleaning, join another table and provide some statistics and insights. Let's go.

## Load the Data
- Importing libraries
- Loading data from the directory in the path `athlete_events.csv`
- Overview

In [9]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('athlete_events.csv')

df.head()

,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


Ok, so far, so good. Let's take a look on data types.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      271116 non-null  int64  
 1   name    271116 non-null  object 
 2   sex     271116 non-null  object 
 3   age     261642 non-null  float64
 4   height  210945 non-null  float64
 5   weight  208241 non-null  float64
 6   team    271116 non-null  object 
 7   noc     271116 non-null  object 
 8   games   271116 non-null  object 
 9   year    271116 non-null  int64  
 10  season  271116 non-null  object 
 11  city    271116 non-null  object 
 12  sport   271116 non-null  object 
 13  event   271116 non-null  object 
 14  medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


Seems like the data has some missing values on height, weight and madal column. Let's look at it in details:

In [11]:
# How many missing values?
df.isna().sum()

id             0
name           0
sex            0
age         9474
height     60171
weight     62875
team           0
noc            0
games          0
year           0
season         0
city           0
sport          0
event          0
medal     231333
dtype: int64

When it come to height and weight the reason for missing data is that this dataset reaches the beginning of olympics journey. The missing values is certainly due to lack collecting data of the competitors in he first half of XX century.

When it comes to many missing data in `medal` column, the reason is somple - data include information about all sportsman and woman. Not only about those who won medals. The remaining columns with missing values are not of interest to us today.



When exploring it, it looked as though some of the teams had hyphens and backslashes. Let's inspect it more closely by inspecting the unique values of the column. 



In [12]:
# Inspect the team column
df['team'].value_counts().to_frame()

,count
team,
United States,17847
France,11988
Great Britain,11404
Italy,10260
Germany,9326
...,...
Briar,1
Hannover,1
Nan-2,1


The `team` column is messy and sometimes countries are separated with `/` or `-`.

In [13]:
# Split the team column on forward slashes and hyphens
df['team_cleaned'] = df['team'].str.split('[/-]').str[0]

df.head(10)

,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal,team_cleaned
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands
5,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN,Netherlands
6,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands
7,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,"Speed Skating Women's 1,000 metres",NaN,Netherlands
8,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands
9,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,"Speed Skating Women's 1,000 metres",NaN,Netherlands


## Bring in additional data
Lets add some data about world nations to broaden the analysis.

In [14]:
select
	name as country,
	year,
	population
from countries
inner join country_stats using(country_id)

IndentationError: unexpected indent (3779165799.py, line 2)

Nice data aobut the countries! Let's merge it to the Olympics

In [15]:
df2 = df.merge(nations_data, left_on=['team_cleaned', 'year'], right_on=['country', 'year'], how='left')

df2.head()

NameError: name 'nations_data' is not defined

## 🏆&nbsp;&nbsp;Which countries have the most gold medals?
Let's start by calculating and visualizing the number of gold medals won by athletes from different countries.

In [16]:
# Count the number of gold medals earned by a country
gold_count = df2.query('medal == "Gold"').groupby('team_cleaned',as_index=False)['medal'].count()

# Sort the values
gold_count.sort_values('medal', ascending=False, inplace=True)

gold_count.head(3)

NameError: name 'df2' is not defined

Let's visualize this with nice choropleth map, where the country's color is based upon the medal count!

In [17]:
fig = px.choropleth(
    gold_count,
    locations='team_cleaned',
    locationmode='country names',
    color='medal',
    labels={'team_cleaned': 'Country', 'medal': 'Medal Count'},
    title='Number of gold medals by country'
)

fig.show()

NameError: name 'gold_count' is not defined

## 📈&nbsp;&nbsp;How has the number of sports grown over time?

In [18]:
sport_count = df2.groupby(['year','season'], as_index=False)['sport'].nunique()

sport_count

NameError: name 'df2' is not defined

Now it's time for the line plot

In [19]:
fig = px.line(
    sport_count,
    x='year',
    y='sport',
    color='season',
    labels={'year': 'Year', 'sport': 'Sport Count', 'season' : 'Season'},
    title='Count of distinct Sports by Year and Season'
    
)

fig.show()

NameError: name 'sport_count' is not defined

## 👪&nbsp;&nbsp;Which countries had the highest medal count per 10 million people in 2016?

In [20]:
# Calculate event medals
event_medals = df2.query('year == 2016')\
    .groupby(['team_cleaned', 'event', 'medal', 'population'], as_index=False)['medal'].first()

event_medals.head()

NameError: name 'df2' is not defined

Calculate the ratio of medal count to population (divided by 10 million for interpretability).

In [21]:
# Group by the team and population
medal_counts = event_medals.groupby(['team_cleaned', 'population'], as_index=False)['medal'].count()

# Calculate the number of medals per 10000000 people
medal_counts['per_10M'] = medal_counts['medal'] / (medal_counts['population'] / 1000000)

# Sort values and take the top 20 countries
top_countries = medal_counts.sort_values('per_10M', ascending=False).head(20)

top_countries.head()

NameError: name 'event_medals' is not defined

How a bar chart would look like?

In [22]:
fig = px.bar(
    top_countries,
    x='team_cleaned',
    y='per_10M',
    labels={'team_cleaned':'Country', 'per_10M':'Medals per 10 milion'},
    title='Medals per 10 million populatio',
    hover_data=['population','medal']
    
)

fig.show()

NameError: name 'top_countries' is not defined

## Conclusion
After some data cleaning and bringing another data (about countries) now we know that:
- USA has the most gold medals of all time
- Number of disciplines for both winter and summer olympics are growing over time
- Grenada, Bahamas and New Zealand have the most medals per 10 millions of citizens (in 2016) - are those countries the most effective in creating champions? :)